In [ ]:
# finetune_layoutlm_perfect.py
# 100% FIXED: Dynamic padding, no data loss, full SROIE + FUNSD
# Using local weights: Dataset/SROIE2019/layoutlm-base-uncased

import os
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset as HFDataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from tqdm.auto import tqdm
import json
import re

# ================================
# CONFIG
# ================================
PRETRAINED_PATH = "Dataset/SROIE2019/layoutlm-base-uncased"
FUNSD_DIR = "Dataset/FUNSD"
SROIE_TRAIN_DIR = "Dataset/SROIE2019/train"
SROIE_TEST_DIR = "Dataset/SROIE2019/test"
OUTPUT_DIR = "finetuned_layoutlm_final_perfect"
LOG_CSV = os.path.join(OUTPUT_DIR, "training_log.csv")

MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 30
LR = 5e-5
WEIGHT_DECAY = 0.01
VAL_RATIO = 0.15

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Label set
label2id = {
    "O": 0,
    "B-HEADER": 1, "I-HEADER": 2,
    "B-QUESTION": 3, "I-QUESTION": 4,
    "B-ANSWER": 5, "I-ANSWER": 6,
    "B-COMPANY": 7, "I-COMPANY": 8,
    "B-DATE": 9, "I-DATE": 10,
    "B-ADDRESS": 11, "I-ADDRESS": 12,
    "B-TOTAL": 13, "I-TOTAL": 14
}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)

# ================================
# DATASET
# ================================
class DocumentDataset(Dataset):
    def __init__(self, tokenizer):
        self.examples = []
        self.tokenizer = tokenizer

        # FUNSD
        for split in ["training_data", "testing_data"]:
            ann_dir = os.path.join(FUNSD_DIR, split, "annotations")
            if not os.path.exists(ann_dir): continue
            for json_file in os.listdir(ann_dir):
                if not json_file.endswith(".json"): continue
                with open(os.path.join(ann_dir, json_file), "r", encoding="utf-8", errors="ignore") as f:
                    data = json.load(f)
                words, bboxes, labels = [], [], []
                for item in data["form"]:
                    text = item["text"].strip()
                    if not text: continue
                    words.append(text)
                    box = item["box"]
                    bboxes.append([box[0], box[1], box[2], box[3]])
                    lbl = item["label"].lower()
                    if lbl == "header": labels.append("B-HEADER")
                    elif lbl == "question": labels.append("B-QUESTION")
                    elif lbl == "answer": labels.append("B-ANSWER")
                    else: labels.append("O")
                if words:
                    self.examples.append({"words": words, "bboxes": bboxes, "ner_tags": labels})

        # SROIE train + test
        for sroie_dir in [SROIE_TRAIN_DIR, SROIE_TEST_DIR]:
            box_dir = os.path.join(sroie_dir, "box")
            entity_dir = os.path.join(sroie_dir, "entities")
            if not os.path.exists(box_dir): continue
            for txt_file in os.listdir(box_dir):
                if not txt_file.endswith(".txt"): continue
                base = txt_file.replace(".txt", "")
                entity_path = os.path.join(entity_dir, base + ".txt")
                if not os.path.exists(entity_path): continue

                words, bboxes, labels = [], [], []
                with open(os.path.join(box_dir, txt_file), "r", encoding="utf-8", errors="ignore") as f:
                    for line in f:
                        parts = line.strip().split(",", 8)
                        if len(parts) != 9: continue
                        x1, y1, _, _, x3, y3, _, _, text = parts
                        words.append(text.strip())
                        bboxes.append([int(x1), int(y1), int(x3), int(y3)])
                        labels.append("O")

                with open(entity_path, "r", encoding="utf-8", errors="ignore") as f:
                    for line in f:
                        if ":" not in line: continue
                        key, value = line.split(":", 1)
                        key = key.strip().lower()
                        value = value.strip().lower()
                        key_map = {"company": "COMPANY", "date": "DATE", "address": "ADDRESS", "total": "TOTAL"}
                        tag = key_map.get(key)
                        if not tag: continue
                        for i, w in enumerate([ww.lower() for ww in words]):
                            if value in w or w in value:
                                if labels[i] == "O":
                                    labels[i] = f"B-{tag}"
                                else:
                                    labels[i] = f"I-{tag}"

                if words:
                    self.examples.append({"words": words, "bboxes": bboxes, "ner_tags": labels})

    def __len__(self): return len(self.examples)
    def __getitem__(self, idx):
        item = self.examples[idx]
        return {
            "words": item["words"],
            "bboxes": item["bboxes"],
            "ner_tags": item["ner_tags"]
        }

# ================================
# PREPROCESS FUNCTION
# ================================
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_PATH)

def preprocess(example):
    words = example["words"]
    bboxes = example["bboxes"]
    labels = example["ner_tags"]

    encoding = tokenizer(
        words,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        is_split_into_words=True,
        return_tensors="pt"
    )

    word_ids = encoding.word_ids()
    input_labels = []
    input_boxes = []

    prev_wid = None
    for i, wid in enumerate(word_ids):
        if wid is None:
            input_labels.append(-100)
            input_boxes.append([0,0,0,0])
        elif wid == prev_wid:
            input_labels.append(-100)
            input_boxes.append(input_boxes[-1])
        else:
            input_labels.append(label2id.get(labels[wid], 0))
            box = bboxes[wid]
            max_x = max(b[2] for b in bboxes) or 1
            max_y = max(b[3] for b in bboxes) or 1
            input_boxes.append([
                int(box[0] * 1000 / max_x),
                int(box[1] * 1000 / max_y),
                int(box[2] * 1000 / max_x),
                int(box[3] * 1000 / max_y)
            ])
            prev_wid = wid

    return {
        "input_ids": encoding.input_ids.flatten(),
        "attention_mask": encoding.attention_mask.flatten(),
        "bbox": torch.tensor(input_boxes),
        "labels": torch.tensor(input_labels)
    }

# ================================
# LOAD DATA
# ================================
raw_dataset = DocumentDataset(tokenizer)
hf_dataset = HFDataset.from_list([preprocess(ex) for ex in tqdm(raw_dataset, desc="Preprocessing")])

train_size = int(0.85 * len(hf_dataset))
val_size = len(hf_dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(hf_dataset, [train_size, val_size])

# ================================
# MODEL + TRAINER
# ================================
model = AutoModelForTokenClassification.from_pretrained(
    PRETRAINED_PATH,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir=OUTPUT_DIR,
    logging_steps=10,
    save_total_limit=2,
    report_to=[],
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    compute_metrics=lambda p: {
        "accuracy": accuracy_score(p.label_ids[p.label_ids != -100], p.predictions.argmax(-1)[p.label_ids != -100]),
        "f1": f1_score(p.label_ids[p.label_ids != -100], p.predictions.argmax(-1)[p.label_ids != -100], average="macro")
    }
)

# Train
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Training complete! Best model saved to: {OUTPUT_DIR}")

d:\Study\Utility\Anaconda\conda\envs\idisc3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Study\Utility\Anaconda\conda\envs\idisc3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Preprocessing: 100%|██████████| 1172/1172 [00:03<00:00, 307.43it/s]
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at Dataset/SROIE2019/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNo

{'loss': 1.1862, 'grad_norm': 2.0486397743225098, 'learning_rate': 4.9880000000000004e-05, 'epoch': 0.08}


  1%|          | 20/3750 [04:21<13:08:35, 12.69s/it]

{'loss': 0.8544, 'grad_norm': 15.424850463867188, 'learning_rate': 4.974666666666667e-05, 'epoch': 0.16}


  1%|          | 30/3750 [06:35<13:50:37, 13.40s/it]

...

{'loss': 0.0005, 'grad_norm': 0.004230150021612644, 'learning_rate': 1.3333333333333335e-08, 'epoch': 30.0}


                                                      
100%|██████████| 3750/3750 [13:28:24<00:00, 10.33s/it]

{'eval_loss': 0.5209712386131287, 'eval_accuracy': 0.9435983615166474, 'eval_f1': 0.6180341533432248, 'eval_runtime': 70.666, 'eval_samples_per_second': 2.491, 'eval_steps_per_second': 0.311, 'epoch': 30.0}


100%|██████████| 3750/3750 [13:28:50<00:00, 12.94s/it]


{'train_runtime': 48530.1569, 'train_samples_per_second': 0.616, 'train_steps_per_second': 0.077, 'train_loss': 0.05545929155767274, 'epoch': 30.0}
Training complete! Best model saved to: finetuned_layoutlm_final_perfect
